In [1]:
from __future__ import print_function
from future_builtins import map, filter, zip

In [2]:
import tensorflow as tf
import keras
import coremltools
import numpy as np

for name, package in [('tensorflow',tf), ('keras',keras), ('coremltools',coremltools), ('numpy', np)]:
    try:
        print(name + ' v' + package.__version__)
    except AttributeError:
        print(name + ' v?')

Using TensorFlow backend.


tensorflow v1.4.0
keras v2.1.1
coremltools v?
numpy v1.13.3


In [3]:
# Load newline-delimited list of categories

# f = open('quickdraw-categories.txt','r')
f = open('50-categories.txt','r')
categories = [line.rstrip() for line in f]
f.close()

filename_from_category = lambda s: '../../quickdraw/'+s+'.npy'
filenames = list(map(filename_from_category, categories))

In [4]:
# Test that all files exist according to categories
# in quickdraw-categories.txt

import os.path

all_exist = True
for filename in filenames:
    if not os.path.isfile(filename):
        print('file `{}` does not exist'.format(filename))
        all_exist = False
if all_exist:
    print('All {} files found!'.format(len(categories)))

All 50 files found!


In [5]:
image_size = (28,28)
samples_per_category = 10000

In [6]:
def one_hot_vector(index):
    hot = np.zeros(len(categories),dtype=np.int8)
    hot[index] = 1
    return hot

def one_hot_array(index):
    is_column = False
    hot = np.zeros((1,len(categories)),dtype=np.int8)
    hot[0,index] = 1
    if is_column:
        return hot.T
    else:
        return hot

In [7]:
import random

def dataset_generator():
    categories_and_filenames = list(zip(categories, filenames))
    # Shuffle order
    random.shuffle(categories_and_filenames)
    for category_index, (category, filename) in enumerate(categories_and_filenames):
        label_one_hot_encoded = one_hot_array(category_index)
        category_images = np.load(filename)
        n_images = category_images.shape[0]
        sample_indices = random.sample(range(n_images), min(samples_per_category,n_images))
        for image in category_images[sample_indices]:
            image = np.matrix(image)
            image = np.expand_dims(image, axis=1)
            yield (image, label_one_hot_encoded)
        yield None

In [8]:
import random

def make_dataset():
    data = []
    target = []
    for category_index, (category, filename) in enumerate(zip(categories, filenames)):
        label_one_hot_encoded = one_hot_vector(category_index)
        category_images = np.load(filename)
        n_images = category_images.shape[0]
        for image in category_images[:samples_per_category]:
#             image = np.matrix(image)
            data.append(image)
            target.append(label_one_hot_encoded)
    data = np.array(data)
    data = np.expand_dims(data, axis=1)
    target = np.array(target)
    print(data.shape)
    print(target.shape)
    return (data, target)

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Reshape, Dense, BatchNormalization, Dropout, \
                         Conv2D, MaxPooling2D, Activation, Flatten, Lambda

### PARAMS ###
input_shape=image_size
n_filters=[32,64,64,256]
dense_sizes=[128,128]
output_size=len(categories)

batch_normalize = True

layer_activation='relu'
final_activation='softmax'

loss='categorical_crossentropy'
optimizer='rmsprop'
metrics=['categorical_accuracy']

def _make_convolution_layers(model):
    model.add(
        Reshape(
            (1, input_shape[0], input_shape[1]), 
            input_shape=(1, input_shape[0]*input_shape[1])
        )
    )
    
    model.add(
        Lambda(lambda x: x/127.5-1)
    )

    for filters in n_filters:
        # Add a single convolution layer
        model.add(
            Conv2D(
                filters=filters,
                kernel_size=(3,3),
                padding='same',
                activation=layer_activation
            )
        )
        # Add batch normalization to the convolution layer
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
        # Pool the layer
        model.add(
            MaxPooling2D(pool_size=(2,2), data_format='channels_first')
        )

    model.add(Flatten())

def _make_dense_layers(model):
    for size in dense_sizes:
        model.add(
            Dense(size, activation=layer_activation)
        )
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
    model.add(
        Dense(output_size, activation=final_activation)
    )

def one_hot_output(y_hat_raw):
    # predict() outputs floats. We want one-hot in most cases
    y_hat = np.zeros(y_hat_raw.shape, dtype=np.int)
    y_hat[np.arange(y_hat_raw.shape[0]), y_hat_raw.argmax(1)] = 1
    return y_hat    

model = Sequential()

_make_convolution_layers(model)
_make_dense_layers(model)

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics
)

In [10]:
data, target = make_dataset()

(500000, 1, 784)
(500000, 50)


In [12]:
hist = model.fit(
    x=data, 
    y=target, 
    batch_size=32, 
    epochs=20, 
    verbose=1,
    shuffle=True
)

Epoch 1/20
500000/500000 [==============================] - 78s 156us/step - loss: 0.9944 - categorical_accuracy: 0.7329
Epoch 2/20
500000/500000 [==============================] - 77s 155us/step - loss: 0.7357 - categorical_accuracy: 0.8040
Epoch 3/20
500000/500000 [==============================] - 78s 155us/step - loss: 0.6750 - categorical_accuracy: 0.8209
Epoch 4/20
500000/500000 [==============================] - 77s 155us/step - loss: 0.6421 - categorical_accuracy: 0.8299
Epoch 5/20
500000/500000 [==============================] - 78s 155us/step - loss: 0.6196 - categorical_accuracy: 0.8358
Epoch 6/20
500000/500000 [==============================] - 79s 157us/step - loss: 0.6005 - categorical_accuracy: 0.8408
Epoch 7/20
500000/500000 [==============================] - 78s 155us/step - loss: 0.5881 - categorical_accuracy: 0.8442
Epoch 8/20
500000/500000 [==============================] - 77s 155us/step - loss: 0.5781 - categorical_accuracy: 0.8467
Epoch 9/20
500000/500000 [======

In [13]:
model.save('cnn_50_cats_10K_imgs_20_epochs.h5')

In [ ]:
# from itertools import cycle

# samples = samples_per_category * len(categories)

# model.fit_generator(
#     cycle(dataset_generator()), 
#     steps_per_epoch=samples, 
#     epochs=40, 
#     verbose=1, 
#     callbacks=None, 
#     validation_data=None, 
#     validation_steps=None, 
#     class_weight=None, 
#     max_queue_size=10,
#     workers=1, 
#     use_multiprocessing=False, 
#     shuffle=True, 
#     initial_epoch=0
# )

In [ ]:
# https://gist.github.com/phrz/98045738d182dd75d1eed74470b9b2ef
def average_line(data, plot_quartiles=True, color='#016d91'):
    '''
    Given 2D data where columns are samples and rows are classes/time series, 
    plot an average line and surrounding quartile area 
    (like a continuous box plot)
    '''
    averages = np.average(data,axis=0)
    x = np.arange(1, len(averages)+1)
    plt.plot(x, averages, color=color)

    if plot_quartiles:
        lower = np.percentile(data, 25, axis=0)
        upper = np.percentile(data, 75, axis=0)
        plt.fill_between(x, lower, upper, color=color, alpha='0.25', linewidth=0)